# Tensorflow Model

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from time import localtime
import matplotlib.pyplot as plt

In [6]:
data_dir = '../NCAA/'
data_file = data_dir + 'MarchMadnessFeatures.csv'
df_features = pd.read_csv(data_file)

X = df_features.iloc[:,1:]
xDim = np.shape(X)[1]
X= X.values.reshape(-1,xDim)
Y = df_features.Result.values
Y = np.array([Y, -(Y-1)]).T  # The model currently needs one column for each class

from sklearn.model_selection import train_test_split
X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.10, stratify=Y)

print('Feature vector dimension is: %.2f' % xDim)


Feature vector dimension is: 34.00


In [7]:
df_features.tail()

,Result,Seed,Pts,Pos,OffRtg,DefRtg,NetRtg,AstR,TOR,TSP,...,OppTOR,OppTSP,OppeFGP,OppFTAR,OppORP,OppDRP,OppRP,OppMAS,OppSAG,OppPOM
1061,0,2,85.764706,71.296376,120.119440,97.344827,-1.751832,14.856466,11.083939,57.275322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1062,0,4,77.468750,67.500600,114.486827,91.185045,-3.479144,13.247290,9.950028,55.611078,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063,0,7,70.709677,67.216103,105.663162,90.941053,-5.430009,12.982481,9.760026,51.248768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1064,0,3,78.727273,65.708218,119.597560,94.423225,-1.711610,16.966904,12.147879,58.999905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1065,0,1,84.212121,67.414109,124.923084,85.741720,-0.853126,16.553182,10.543421,61.598531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Parameter
learning_rate = 0.0003
training_epochs = 200
batch_size = 50 #25
display_step = 5


# Network Parameters
n_hidden_1 = 200 
n_hidden_2 = 200
n_hidden_3 = 200
n_hidden_4 = 200
n_input = xDim # Number of feature
n_classes = 2 


# Assign placeholders for unknown values
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with sigmoif activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Hidden layer with sigmoif activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h2']), biases['b2'])
    layer_3 = tf.nn.sigmoid(layer_3)
    # Hidden layer with sigmoif activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h2']), biases['b2'])
    layer_4 = tf.nn.sigmoid(layer_4)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


# Initializing the variables
init = tf.global_variables_initializer()

In [4]:
# Launch the graph
logloss = []
acc = []
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X)/batch_size)
        X_batches = np.array_split(X, total_batch)
        Y_batches = np.array_split(Y, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            logloss.append(avg_cost)
            correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            # Calculate accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            acc.append( accuracy.eval({x: X_test, y: Y_test}))
    print("Optimization Finished!")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))
    global result 
    result = tf.argmax(pred, 1).eval({x: X_test, y: Y_test})

Epoch: 0001 cost= nan
Epoch: 0006 cost= nan
Epoch: 0011 cost= nan
Epoch: 0016 cost= nan
Epoch: 0021 cost= nan
Epoch: 0026 cost= nan
Epoch: 0031 cost= nan
Epoch: 0036 cost= nan
Epoch: 0041 cost= nan
Epoch: 0046 cost= nan
Epoch: 0051 cost= nan
Epoch: 0056 cost= nan
Epoch: 0061 cost= nan
Epoch: 0066 cost= nan


KeyboardInterrupt: 

In [6]:
plt.plot(logloss)
plt.plot(acc)

In [16]:
tf.reset_default_graph

<function tensorflow.python.framework.ops.reset_default_graph>